In [49]:
from opensearchpy import OpenSearch

In [50]:
#Initialize connection to opensearch
host = 'localhost'
port = 9200
auth = ('admin', 'admin') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    timeout=100
)
#check status
print(client.info())

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'name': 'opensearch-node1_med', 'cluster_name': 'opensearch-cluster', 'cluster_uuid': 'DbPN66O6QdGNwomjb1ufRQ', 'version': {'distribution': 'opensearch', 'number': '2.11.1', 'build_type': 'tar', 'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752', 'build_date': '2023-11-29T21:43:10.135035992Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


In [51]:
indices = client.indices.get_alias().keys()

# Print the indices
for index in indices:
    print(index)

security-auditlog-2024.02.26
.opensearch-observability
.plugins-ml-config
.opensearch-sap-log-types-config
.opendistro_security
voyage-2-large


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
 response = client.indices.delete(index="security-auditlog-2024.02.26")

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [67]:
#create VectorDB index:

index_name = "text-embedding-3-large"
index_body = {
    "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": {
    "properties": {
        "vector": {
          "type": "knn_vector",
          "dimension": 1024,     #Thats the output dimension of the model
          "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
          }
        }
    }
  }
}

response = client.indices.create(index_name, body = index_body)
print(response)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'text-embedding-3-large'}


Create Infrastructure for hybrid search

In [ ]:
url = "/_search/pipeline/nlp_search-pipeline"
hybrid_search_pipeline ={
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.5,
              0.5
            ]
          }
        }
      }
    }
  ]
}
#client.pipeline.create("hybrid_search_pipeline", body=nlp_search_pipeline)
res = client.transport.perform_request(method = "PUT", url = url, body = hybrid_search_pipeline) 

In [ ]:

route = "/{index}}/_search?search_pipeline=nlp_search-pipeline"
hybrid_search_body = {
  "_source": {
    "exclude": [
      "vector"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "text": {
              "query": "query"
            }
          }
        },
        {
          "knn": {                  #instead of neural do knn search since we already have our embeddings
            "vector": {
              "vector": "query_embedding",
              "k": 5
            }
          }
        }
      ]
    }
  }
}
#response = client.search(index="wmt_voyage-large-2-fixed", body=hybrid_search_body, search_type="query_then_fetch", pipeline = hybrid_search_pipeline,)
ress  = client.transport.perform_request(method = "GET", url = route, body = hybrid_search_body) 

In [82]:
data = []

In [85]:

index_name = "voyage-2-large"
import csv
from tqdm import tqdm

# Open the CSV file in read mode
with open("voyage-lare-2.csv", "r", encoding = "utf-8") as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    for row in reader:
        data.append(row)

In [89]:
evl_data = []

for entry in tqdm(data[0]):
    evl_data.append(eval(entry))

100%|██████████| 57560/57560 [07:12<00:00, 133.12it/s]


In [99]:
len(evl_data[5003][0]) # 0:vector 1:pmid 2:text 3:cite 4:id

1536

In [66]:
def divide_list(input_list, n):
    chunk_size = len(input_list) // n
    remainder = len(input_list) % n

    start = 0
    result = []

    for i in range(n):
        end = start + chunk_size + (1 if i < remainder else 0)
        result.append(input_list[start:end])
        start = end

    return result


In [97]:
result = divide_list(evl_data, 600)

In [98]:
len(result[599])

95

In [100]:
index_name = "voyage-2-large"
import uuid
from tqdm import tqdm
data_for_bulk_insert = []

for batch in tqdm(result):
    for chunk in batch:
        data_for_bulk_insert.append({"index": {"_index": index_name, "_id": str(uuid.uuid4())}})
        data_for_bulk_insert.append({"vector" : chunk[0], "PMID" : chunk[1], "text" : chunk[2], "cite" : chunk[3]})
    response = client.bulk(data_for_bulk_insert)
    data_for_bulk_insert = []

  0%|          | 0/600 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  0%|          | 1/600 [00:00<07:27,  1.34it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  0%|          | 2/600 [00:01<05:54,  1.69it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundat